# Content Creation at Scale

## Initial Imports

In [1]:
# warning control
import warnings
warnings.filterwarnings("ignore")

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

import litellm
litellm.set_verbose = True
os.environ['LITELLM_LOG'] = 'DEBUG'

## Creating Structred Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    """
    Model for social media post
    """
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, Linkedin).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    """
    Model for content output
    """
    article: str = Field(..., description="The article formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")


## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml',
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [6]:
from crewai import LLM


mistral_model = 'mistral/mistral-large-latest'
# groq_llm = "groq/llama-3.3-70b-versatile"
groq_llm = "groq/mistral-saba-24b"

tools_llm_configs = {
    "small": {
        "provider":"mistralai",
        "config":{
            "model":"mistral/mistral-small-latest",
            'api_key': os.getenv("MISTRAL_API_KEY"),
        },
    },
    "large": {
        "provider":"mistralai",
        "config":{
            "model":"mistral/mistral-large-latest",
            'api_key': os.getenv("MISTRAL_API_KEY"),
        }},
}

llms = {
    "small": LLM(
        model="mistral/mistral-small-latest",
        provider="mistralai",
        config={
            "api_key": os.getenv("MISTRAL_API_KEY")
        }
    ),
    "large": LLM(
        model="mistral/mistral-large-latest",
        provider="mistralai",
        config={
            "api_key": os.getenv("MISTRAL_API_KEY")
        }
    ),
}

## Creating Crews, Agents and Tasks

### Creating Agents

In [7]:
def create_website_search_tool(llm_config):
    return WebsiteSearchTool(config={
        "llm": llm_config,
        "embedder": {
            "provider": "mistralai",
            "config": {
                "model": "mistral-embed",
                "api_key": os.getenv("MISTRAL_API_KEY")
            }
        }
    })

In [8]:
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=mistral_model,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[
        SerperDevTool(), 
        WebsiteSearchTool(
            config={
                "llm": {
                    "config":{
                    "model": "mistral/mistral-small-latest",
                    "api_key": os.getenv("MISTRAL_API_KEY")
                    },
                },
                "embedder": {
                "config": {
                    "model": "mistral-embed",
                    "api_key": os.getenv("MISTRAL_API_KEY")
                    }
                }
            }
        )],
llm=mistral_model,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[
        SerperDevTool(), 
        WebsiteSearchTool(
            config={
                "llm": {
                    "config":{
                    "model": "mistral/mistral-small-latest",
                    "api_key": os.getenv("MISTRAL_API_KEY")
                    },
                },
                "embedder": {
                "config": {
                    "model": "mistral-embed",
                    "api_key": os.getenv("MISTRAL_API_KEY")
                    }
                }
            }
        )
        ],
    llm=mistral_model
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
    llm=mistral_model
)

### Creating Tasks

In [9]:

monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

### Creating Crew

In [10]:
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
        ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

In [11]:
from litellm import completion
import os

os.environ["LITELLM_DEBUG"] = "True"  # Enable debug logging

models = {
    "small": "mistral/mistral-small-latest",
    "large": "mistral/mistral-large-latest"
}

for name, model in models.items():
    print(f"\n🔍 Testing {name} model: {model}")
    try:
        response = completion(
            model=model,
            api_key=os.getenv("MISTRAL_API_KEY"),
            messages=[{"role": "user", "content": "Say hello in 3 words"}]
        )
        content = response["choices"][0]["message"]["content"]
        print(f"✅ {name.capitalize()} Model Response: {content}")
    except Exception as e:
        print(f"❌ {name.capitalize()} Model Error:", str(e))

15:58:59 - LiteLLM:WARNING: utils.py:428 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
2025-04-28 15:58:59,457 - 37940 - utils.py-utils:428 - WARNING: `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.



🔍 Testing small model: mistral/mistral-small-latest
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'extra_body': {}}


15:58:59 - LiteLLM:WARNING: utils.py:428 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
2025-04-28 15:58:59,942 - 37940 - utils.py-utils:428 - WARNING: `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "8e284e7a034f4122aba0221559812d28", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "Hello! How are you?", "refusal": null, "role": "assistant", "annotations": null, "audio": null, "function_call": null, "tool_calls": null}}], "created": 1745836137, "model": "mistral-small-latest", "object": "chat.completion", "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 7, "prompt_tokens": 9, "total_tokens": 16, "completion_tokens_details": null, "prompt_tokens_details": null}}


✅ Small Model Response: Hello! How are you?

🔍 Testing large model: mistral/mistral-large-latest
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'extra_body': {}}
RAW RESPONSE:
{"id": "8baeba2ab7f94b758242eb5d49e9bc56", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "Hello, how are?", "refusal": null, "role": "a

## Kicking off the Crew

In [12]:
result = content_creation_crew.kickoff(inputs={
    'subject': 'Gold rates in India',
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1129d46b-9660-44ab-9d33-1e2cf87ac17e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 650e0152-51d9-4282-a55b-611b379a85a1
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 650e0152-51d9-4282-a55b-611b379a85a1
       Status: Executing Task...
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Gold rates in India. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Gold rates in India.



🚀 Crew: crew
└── 📋 Task: 650e0152-51d9-4282-a55b-611b379a85a1
       Status: Executing Task...
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress
        └── 🧠 Thinking...

...

16:04:21 - LiteLLM:WARNING: utils.py:428 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
2025-04-28 16:04:21,514 - 37940 - utils.py-utils:428 - WARNING: `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "dac32ba094504e449122a1eabaa0c18a", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "### Detailed Summary Report on Gold Rates in India\n\n#### Current Gold Rates and Trends\n\n**Forbes (April 28, 2025):**\nThe price of gold (XAU) as of April 28, 2025, is INR 9,027 per gram of 24-carat gold, marking a 0.71% decrease from the previous day's close of INR 9,091. Over the past week, gold has declined by 1.72%, although it has risen by 7.21% compared to a month ago. The 52-week high for gold prices stands at INR 9,410, while the 52-week low is INR 8,101. The price per gram of 22-carat gold is currently INR 8,274. Gold prices vary by city, with detailed rates available for major cities across India.\n\n**The Hindu Business Line:**\nAs of April 28, 2025, the gold price in India is \u20b99,015 per gram for 22-carat gold and \u20b99,466 per gram for 24-carat gold. Gold rates have shown a steady trend since the start of the year, wi

🚀 Crew: crew
├── 📋 Task: 650e0152-51d9-4282-a55b-611b379a85a1
│      Assigned to: Lead Market Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Lead Market Analyst
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (1)
│       └── 🔧 Used Read website content (4)
├── 📋 Task: f98474f3-36bb-4ccc-bf2b-60c793fd2739
│      Assigned to: Chief Data Strategist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chief Data Strategist
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Failed Search in a specific website (3)
├── 📋 Task: 2e5b4a3e-87e6-4e73-aa8f-a9b2299745b1
│      Assigned to: Creative Content Director
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Content Director
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (4)
│       └── 🔧 Failed Search in a specific website (4)
└── 📋 Task: a1aa8959-d499-4a88-8735-9ff73123bfee
       Assigned to: Chief Content Officer
    
       Status: ✅ Completed
    └── 🤖 Agent: Chief Content Officer
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a1aa8959-d499-4a88-8735-9ff73123bfee                                                                     │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1129d46b-9660-44ab-9d33-1e2cf87ac17e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Social Content

In [13]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
💥 **Gold Rates Alert!** 💥 As of April 28, 2025,
24-carat gold is at INR 9,027/gram. 🌟 Key
insights: - Down 0.71% from yesterday - Up 7.21%
from last month - 52-week high: INR 9,410 -
52-week low: INR 8,101 Stay tuned for more
updates! [#GoldRates](https://twitter.com/hashtag/
GoldRates) [#InvestmentTips](https://twitter.com/h
ashtag/InvestmentTips) [#FinancialNews](https://tw
itter.com/hashtag/FinancialNews)
--------------------------------------------------
LinkedIn
🌟 **Gold Market Update: April 28, 2025** 🌟 Gold
prices continue to fluctuate, with 24-carat gold
at INR 9,027 per gram. Key factors influencing
prices include central bank reserves, USD
strength, and demand dynamics. 📈 **Investment
Tip**: Consider diversifying with Sovereign Gold
Bonds (SGBs) and Gold ETFs for better returns and
lower costs. Read our latest blog for a detailed
analysis: [Link to Blog
Post](https://www.example.com/blog-post) [#GoldInv
estment](https://www.linkedin.com/feed/hashtag/gol
dinvestment/) 

C:\Users\ZTI\AppData\Local\Temp\ipykernel_34100\3812343593.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  posts = result.pydantic.dict()['social_media_posts']


## Blog Post

In [14]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

C:\Users\ZTI\AppData\Local\Temp\ipykernel_34100\3784749904.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  display(Markdown(result.pydantic.dict()['article']))


## Detailed Summary Report on Gold Rates in India

### Current Gold Rates and Trends

#### Forbes (April 28, 2025):

The price of gold (XAU) as of April 28, 2025, is INR 9,027 per gram of 24-carat gold, marking a 0.71% decrease from the previous day's close of INR 9,091. Over the past week, gold has declined by 1.72%, although it has risen by 7.21% compared to a month ago. The 52-week high for gold prices stands at INR 9,410, while the 52-week low is INR 8,101. The price per gram of 22-carat gold is currently INR 8,274. Gold prices vary by city, with detailed rates available for major cities across India.

#### The Hindu Business Line:

As of April 28, 2025, the gold price in India is ₹9,015 per gram for 22-carat gold and ₹9,466 per gram for 24-carat gold. Gold rates have shown a steady trend since the start of the year, with most demand coming from jewelry rather than gold biscuits and coins. This trend reflects the cultural significance and investment preferences in India.

#### Economic Times (Morningstar Analysis):

Gold prices have reached record highs, with 24-carat gold nearing Rs 90,000 per 10 grams in Indian markets and surpassing $3,100 in global markets. However, a Morningstar analyst predicts a potential 38% drop in gold prices, which could bring the price down to around Rs 55,000 per 10 grams in India. Factors driving this potential price drop include increased supply, declining demand, and market saturation.

### Factors Influencing Gold Prices

#### Central Bank Gold Reserves:

Gold is considered an important reserve for central banks worldwide. Increased purchases by central banks tend to drive up gold prices. The Indian government recently hiked the import duty on gold from 10.75% to 15% to curb gold imports and preserve foreign reserves. This move aims to control the outflow of foreign currency and stabilize the economy.

#### Strength of the U.S. Dollar:

The price of gold is inversely proportional to the value of the U.S. dollar. A stronger U.S. dollar generally leads to lower gold prices. In India, the USD to INR conversion rate significantly impacts gold pricing, making it crucial for investors to monitor currency fluctuations.

#### Demand for Gold:

Household and industrial demand for gold, particularly during festivals and special occasions, pushes prices higher. Additionally, investment demand, driven by ETFs and other financial instruments, plays a crucial role in gold price movements. Festivals like Diwali, Dhanteras, and Akshaya Tritiya see a surge in gold purchases, reflecting cultural and investment sentiments.

#### Interest Rates:

Higher interest rates increase the opportunity cost of holding non-interest-bearing assets like gold, leading to a decrease in gold prices. Conversely, lower interest rates can make gold more attractive as an investment, driving up its price. Central bank policies on interest rates thus have a direct impact on gold pricing.

#### Gold Production Capacity:

Increased gold production and recycling activity can lead to a higher supply, which may put downward pressure on gold prices if demand remains unchanged. Efficient mining and recycling practices can influence the market dynamics significantly.

#### Inflation:

Gold is often considered a hedge against inflation. During periods of high inflation, investors turn to gold as a safe-haven asset, driving up demand and prices. This makes gold a popular investment choice during economic uncertainty.

#### Festivals and Cultural Events:

In India, gold prices often surge during festive seasons like Diwali, Dhanteras, and Akshaya Tritiya due to increased demand for gifting and investment purposes. These cultural events play a pivotal role in the gold market, driving both demand and prices.

### Investment Options and Market Dynamics

#### Physical Gold:

Physical gold can be purchased as jewelry, coins, or bars. However, it comes with additional costs such as making charges and GST. Hallmarked jewelry ensures purity and value for money, making it a reliable investment option despite the extra costs.

#### Digital Gold:

Digital gold investments include Sovereign Gold Bonds (SGBs), Gold ETFs, and digital gold wallets. SGBs offer an assured interest rate of 2.5% semi-annually and are considered better investments than physical gold due to lower costs and better returns. Gold ETFs provide easy entry and exit points and are traded like mutual fund units on stock exchanges, offering liquidity and flexibility.

#### Gold Import Duty:

Import duty on gold jewelry, bars, and coins is currently set at 12.5%. Including GST, the final tax on physical gold assets turns out to be 12.5% + 3% flat GST + 5% GST on making charges. Imports above 1 kg are taxed at 36.05%, with additional GST, reflecting the government's efforts to control gold imports and manage foreign reserves.

### Future Outlook and Analyst Predictions

#### Morningstar Analysis:

Despite the recent rally, increased supply and declining demand could drive gold prices down by nearly 40% in the coming years. This prediction is based on market saturation and potential corrections in the gold sector.

#### Market Saturation and ETFs:

Mergers and acquisitions in the gold sector, along with a surge in gold-backed ETFs, suggest a peak market and potential price corrections. These market dynamics indicate that investors should be cautious and consider diversifying their portfolios.

### Conclusion

The gold market in India is influenced by a multitude of factors, including central bank policies, currency fluctuations, demand dynamics, and economic indicators. While gold prices have reached record highs, analysts predict potential declines due to increased supply and market saturation. Investors should consider diversifying their portfolios with digital gold options like SGBs and Gold ETFs, which offer better returns and lower costs compared to physical gold.

For further detailed analysis and investment strategies, please consult with financial experts and portfolio managers.

---

## Blog Post: Understanding Gold Rates in India - Trends, Insights, and Investment Strategies

Gold has always held a special place in Indian culture and economics. Whether it's for investment, jewelry, or cultural significance, gold remains a hot topic. Let's delve into the current gold rates, trends, and investment strategies to help you make informed decisions.

### Current Gold Rates and Trends

As of April 28, 2025, the price of gold (XAU) is INR 9,027 per gram of 24-carat gold, down 0.71% from the previous day's close of INR 9,091. Compared to last week, gold is down 1.72%, but it is up 7.21% from a month ago. The 52-week gold price high is INR 9,410, while the 52-week gold price low is INR 8,101. The price per gram of 22-carat gold is INR 8,274.

According to **The Hindu Business Line**, gold rates have witnessed a steady trend since the start of the year, with most demand coming from jewelry rather than gold biscuits and coins.

### Factors Influencing Gold Prices

Several factors influence gold prices in India:

#### Central Bank Gold Reserves:

Central banks worldwide consider gold an important reserve. Increased purchases by central banks tend to drive up gold prices. The Indian government recently hiked import duty on gold from 10.75% to 15% to curb gold imports and preserve foreign reserves.

#### Strength of the U.S. Dollar:

The price of gold is inversely proportional to the value of the U.S. dollar. A stronger U.S. dollar generally leads to lower gold prices. In India, the USD to INR conversion rate significantly impacts gold pricing.

#### Demand for Gold:

Household and industrial demand for gold, particularly during festivals and special occasions, pushes prices higher. Investment demand, driven by ETFs and other financial instruments, also plays a crucial role in gold price movements.

#### Interest Rates:

Higher interest rates increase the opportunity cost of holding non-interest-bearing assets like gold, leading to a decrease in gold prices. Conversely, lower interest rates can make gold more attractive as an investment, driving up its price.

#### Gold Production Capacity:

Increased gold production and recycling activity can lead to a higher supply, which may put downward pressure on gold prices if demand remains unchanged.

#### Inflation:

Gold is often considered a hedge against inflation. During periods of high inflation, investors turn to gold as a safe-haven asset, driving up demand and prices.

#### Festivals and Cultural Events:

In India, gold prices often surge during festive seasons like Diwali, Dhanteras, and Akshaya Tritiya due to increased demand for gifting and investment purposes.

### Investment Options and Market Dynamics

Investors have various options for investing in gold:

#### Physical Gold:

Physical gold can be purchased as jewelry, coins, or bars. However, it comes with additional costs such as making charges and GST. Hallmarked jewelry ensures purity and value for money.

#### Digital Gold:

Digital gold investments include Sovereign Gold Bonds (SGBs), Gold ETFs, and digital gold wallets. SGBs offer an assured interest rate of 2.5% semi-annually and are considered better investments than physical gold due to lower costs and better returns. Gold ETFs provide easy entry and exit points and are traded like mutual fund units on stock exchanges.

#### Gold Import Duty:

Import duty on gold jewelry, bars, and coins is currently set at 12.5%. Including GST, the final tax on physical gold assets turns out to be 12.5% + 3% flat GST + 5% GST on making charges. Imports above 1 kg are taxed at 36.05%, with additional GST.

### Future Outlook and Analyst Predictions

Despite the recent rally, increased supply and declining demand could drive gold prices down by nearly 40% in the coming years. Major financial institutions like Bank of America and Goldman Sachs remain optimistic, predicting higher gold prices in the next two years.

Mergers and acquisitions in the gold sector, along with a surge in gold-backed ETFs, suggest a peak market and potential price corrections.

### Conclusion

The gold market in India is influenced by a multitude of factors, including central bank policies, currency fluctuations, demand dynamics, and economic indicators. While gold prices have reached record highs, analysts predict potential declines due to increased supply and market saturation. Investors should consider diversifying their portfolios with digital gold options like SGBs and Gold ETFs, which offer better returns and lower costs compared to physical gold.

For further detailed analysis and investment strategies, please consult with financial experts and portfolio managers.

---

### Social Media Updates

#### Twitter

💥 **Gold Rates Alert!** 💥 As of April 28, 2025, 24-carat gold is at INR 9,027/gram. 🌟 Key insights: - Down 0.71% from yesterday - Up 7.21% from last month - 52-week high: INR 9,410 - 52-week low: INR 8,101 Stay tuned for more updates! [#GoldRates](https://twitter.com/hashtag/GoldRates) [#InvestmentTips](https://twitter.com/hashtag/InvestmentTips) [#FinancialNews](https://twitter.com/hashtag/FinancialNews)

#### LinkedIn

🌟 **Gold Market Update: April 28, 2025** 🌟 Gold prices continue to fluctuate, with 24-carat gold at INR 9,027 per gram. Key factors influencing prices include central bank reserves, USD strength, and demand dynamics. 📈 **Investment Tip**: Consider diversifying with Sovereign Gold Bonds (SGBs) and Gold ETFs for better returns and lower costs. Read our latest blog for a detailed analysis: [Link to Blog Post](https://www.example.com/blog-post) [#GoldInvestment](https://www.linkedin.com/feed/hashtag/goldinvestment/) [#FinancialMarkets](https://www.linkedin.com/feed/hashtag/financialmarkets/) [#EconomicInsights](https://www.linkedin.com/feed/hashtag/economicinsights/)

#### Instagram

📸 **Gold Rates Today** 📸 [Image of gold bars and coins] 24-carat gold: INR 9,027/gram ✨ 22-carat gold: INR 8,274/gram ✨ 💡 **Did You Know?** Gold prices are influenced by central bank policies, USD strength, and festive demand. Stay informed and invest smartly! 💼 [#GoldRates](https://www.instagram.com/explore/tags/goldrates/) [#InvestmentGoals](https://www.instagram.com/explore/tags/investmentgoals/) [#FinancialLiteracy](https://www.instagram.com/explore/tags/financialliteracy/)

#### Facebook

🌟 **Gold Market Insights: April 28, 2025** 🌟 Gold prices are currently at INR 9,027 per gram for 24-carat gold. Key factors driving prices include central bank reserves, USD strength, and demand during festivals. 📈 **Expert Tip**: Diversify your portfolio with digital gold options like SGBs and Gold ETFs for better returns. Read our comprehensive blog for more details: [Link to Blog Post](https://www.example.com/blog-post) [#GoldInvestment](https://www.facebook.com/hashtag/goldinvestment) [#FinancialTips](https://www.facebook.com/hashtag/financialtips) [#MarketUpdates](https://www.facebook.com/hashtag/marketupdates)

In [83]:
import os

article_content = result.pydantic.dict()['article']
file_name = "generated_blog.md"

with open(file_name, "w", encoding="utf-8") as f:
    f.write(article_content)

print(f"✅ Blog saved as {file_name}")

✅ Blog saved as generated_blog.md


In [87]:
blog_post = result['article'].split('## Blog Post: ')[1]
print(blog_post)

Navigating the US-China Tariff Landscape: Insights and Market Reactions

### Introduction:
The recent implementation of US tariffs on China has sent shockwaves through global markets, leading to significant volatility and geopolitical tensions. As China retaliates with its own set of tariffs, the trade war between the two economic giants continues to escalate. This blog post delves into the key findings and market reactions, providing investors and businesses with actionable insights to navigate the evolving trade landscape.

### Market Volatility and Sentiment:
The announcement of a 90-day pause on tariffs for most countries, excluding China, has resulted in substantial market volatility. China's retaliatory tariffs, raising levies on US goods to 84%, have exacerbated market instability. Investor sentiment has been negatively impacted, with concerns over disrupted supply chains, increased inflation, and potential economic slowdown.

Market volatility has been particularly pronounced i